## Budget Optimization in Healthcare Scenario

| **File Name**      | **Description**                                      | **Dimensions**       |
|---------------------|------------------------------------------------------|----------------------|
| `Ujt.csv`          | Cost for employee assignments                        | (30 x 8)            |
| `Vqt.csv`          | Costs for patient assignments                        | (200 x 8)           |
| `Wst.csv`          | Costs for resource allocation                        | (15 x 8)            |
| `Xqt.csv`          | Penalties for unserved patients                      | (200 x 8)           |
| `Yj.csv`           | Overtime costs for employee                            | (30 x 1)            |
| `Hj.csv`           | Maximum working hours for employee                      | (30 x 1)            |
| `Zt.csv`           | Demand for staff per time slot                       | (8 x 1)             |
| `Budget.csv`       | Total budget                                         | (1 x 1)             |


In [ ]:
!pip install pyomo
from pyomo.environ import *
import pandas as pd
import numpy as np
!apt-get install -y -qq glpk-utils

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.7 MB/s eta 0:00:00
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 123630 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_5.0-1_amd64.deb ...
Unpacking libglpk40:amd64 (5.0-1) ...
Selecting previously unselected package

In [ ]:
Ujt = pd.read_csv('/content/drive/MyDrive/Optimization Project/Ujt.csv', index_col=0).values  # Employee cost
Vqt = pd.read_csv('/content/drive/MyDrive/Optimization Project/Vqt.csv', index_col=0).values  # Patient cost
Wst = pd.read_csv('/content/drive/MyDrive/Optimization Project/Wst.csv', index_col=0).values  # Resource cost
Xqt = pd.read_csv('/content/drive/MyDrive/Optimization Project/Xqt.csv', index_col=0).values  # Penalty
Yj = pd.read_csv('/content/drive/MyDrive/Optimization Project/Yj.csv', index_col=0)['Yj'].values  # Overtime weight
Hj = pd.read_csv('/content/drive/MyDrive/Optimization Project/Hj.csv', index_col=0)['Hj'].values  # Max working hours
Zt = pd.read_csv('/content/drive/MyDrive/Optimization Project/Zt.csv', index_col=0)['Zt'].values  # Demand
B = pd.read_csv('/content/drive/MyDrive/Optimization Project/Budget_updated.csv', index_col=0)['B'].iloc[0]  # Budget

In [ ]:
from pyomo.environ import *

# Indices
N,T = Ujt.shape  # Number of employees, time slots
P = Vqt.shape[0]  # Number of patients
R = Wst.shape[0]  # Number of resources

# Create Pyomo model
model = ConcreteModel()

# Sets
model.employees = RangeSet(0,N-1)  # Employee indices
model.patients = RangeSet(0,P-1)   # Patient indices
model.resources = RangeSet(0,R-1)  # Resource indices
model.times = RangeSet(0,T-1)      # Time slot indices

# Parameters
model.Ujt = Param(model.employees, model.times, initialize=lambda model, j, t: Ujt[j,t])
model.Vqt = Param(model.patients, model.times, initialize=lambda model, q, t: Vqt[q,t])
model.Wst = Param(model.resources, model.times, initialize=lambda model, s, t: Wst[s,t])
model.Xqt = Param(model.patients, model.times, initialize=lambda model, q, t: Xqt[q,t])
model.Yj = Param(model.employees, initialize=lambda model, j: Yj[j])
model.Hj = Param(model.employees, initialize=lambda model, j: Hj[j])
model.Zt = Param(model.times, initialize=lambda model, t: Zt[t])
model.B = Param(initialize=B)

# Decision Variables
model.a = Var(model.employees, model.times, domain=Binary)  # Employee assignment
model.b = Var(model.patients, model.times, domain=Binary)   # Patient scheduling
model.c = Var(model.resources, model.times, domain=Binary)  # Resource allocation
model.d = Var(model.employees, domain=Binary)               # Overtime

# Objective Function
def objective_rule(model):
    return (
        sum(model.Ujt[j,t] * model.a[j,t] for j in model.employees for t in model.times) +
        sum(model.Vqt[q,t] * model.b[q,t] for q in model.patients for t in model.times) +
        sum(model.Wst[s,t] * model.c[s,t] for s in model.resources for t in model.times) +
        sum(model.Xqt[q,t] * ( model.b[q,t]) for q in model.patients for t in model.times) +
        sum(model.Yj[j] * model.d[j] for j in model.employees)
    )
model.objective = Objective(rule=objective_rule,sense=minimize)

# Constraints
# The total number of employees assigned to each time slot must equal the demand for employees at that time.
def employee_availability_rule(model,t):
    return sum(model.a[j,t] for j in model.employees) == model.Zt[t]
model.employee_availability = Constraint(model.times, rule=employee_availability_rule)

# The total number of time slots each employee can be assigned to is limited by their maximum working hours.
def max_working_hours_rule(model,j):
    return sum(model.a[j,t] for t in model.times) <= model.Hj[j]
model.max_working_hours = Constraint(model.employees, rule=max_working_hours_rule)

# Each patient is assigned to exactly one time slot
def patient_scheduling_rule(model,q):
    return sum(model.b[q,t] for t in model.times) == 1
model.patient_scheduling = Constraint(model.patients, rule=patient_scheduling_rule)

# Each resource is allocated to exactly one time slot
def resource_allocation_rule(model,s):
    return sum(model.c[s,t] for t in model.times) == 1
model.resource_allocation = Constraint(model.resources, rule=resource_allocation_rule)

# If an employee works overtime, the sum of their assigned time slots must exceed their maximum working hours
def overtime_rule(model,j):
    return model.d[j] >= sum(model.a[j,t] for t in model.times) - model.Hj[j]
model.overtime = Constraint(model.employees, rule=overtime_rule)

# The total cost of employee assignments and resource allocations must not exceed the budget constraint
def budget_rule(model):
    return (
        sum(model.Ujt[j,t] * model.a[j,t] for j in model.employees for t in model.times) +
        sum(model.Yj[j] * model.d[j] for j in model.employees) +
        sum(model.Wst[s,t] * model.c[s,t] for s in model.resources for t in model.times)
    ) <= model.B
model.budget = Constraint(rule=budget_rule)

# Solve the model
solver = SolverFactory('glpk', executable='/usr/bin/glpsol')
results = solver.solve(model)

# Exporting Results
# Employee Assignments
employee_assignment = []
for j in model.employees:
    for t in model.times:
        if model.a[j,t].value > 0.5:
            employee_assignment.append({"Employee": j + 1, "TimeSlot": t + 1})
employee_assignment_df = pd.DataFrame(employee_assignment)
employee_assignment_df.to_csv("/content/drive/MyDrive/Optimization Project/employee_assignment.csv", index=False)

# Patient Scheduling
patient_schedule = []
for q in model.patients:
    for t in model.times:
        if model.b[q,t].value > 0.5:
            patient_schedule.append({"Patient": q + 1, "TimeSlot": t + 1})
patient_schedule_df = pd.DataFrame(patient_schedule)
patient_schedule_df.to_csv("/content/drive/MyDrive/Optimization Project/patient_scheduling.csv", index=False)

# Resource Allocation
resource_allocation = []
for s in model.resources:
    for t in model.times:
        if model.c[s, t].value > 0.5:
            resource_allocation.append({"Resource": s + 1, "TimeSlot": t + 1})
resource_allocation_df = pd.DataFrame(resource_allocation)
resource_allocation_df.to_csv("/content/drive/MyDrive/Optimization Project/resource_allocation.csv", index=False)

print(model.objective())


5452.0


In [ ]:
print(model.a[1,4]())

1.0


In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/Optimization Project/employee_assignment.csv')
df1

,Employee,TimeSlot
0,1,2
1,1,3
2,1,5
3,1,6
4,1,7
...,...,...
121,29,8
122,30,2
123,30,6
124,30,7


In [ ]:
df2 = pd.read_csv('/content/drive/MyDrive/Optimization Project/patient_scheduling.csv')
df2

,Patient,TimeSlot
0,1,4
1,2,8
2,3,5
3,4,8
4,5,3
...,...,...
195,196,1
196,197,2
197,198,6
198,199,2


In [ ]:
df3 = pd.read_csv('/content/drive/MyDrive/Optimization Project/resource_allocation.csv')
df3

,Resource,TimeSlot
0,1,6
1,2,8
2,3,3
3,4,8
4,5,3
5,6,5
6,7,1
7,8,4
8,9,6
9,10,8


Resources:  

1. Hospital beds  
2. Operating rooms  
3. Medical equipment  
4. Laboratory facilities  
5. Radiology equipment  
6. Surgical instruments  
7. Medications  
8. Oxygen Tanks  
9. Intensive care unit (ICU) beds  
10. Anesthesia machines  
11. Patient monitoring systems  
12. Wheelchairs  
13. Surgical supplies  
14. Imaging and diagnostic equipment  
15. Personal protective equipment (PPE)  

In [ ]:
# @title Default title text
# # Synthetic Data Generation

# # Parameters for input data
# N = 10  # Number of staff members
# P = 15  # Number of patients
# R = 5   # Number of resources
# T = 8   # Number of time slots

# # Generate random weights and costs
# Wit = np.random.randint(1, 10, size=(N, T))  # Weight for staff assignments
# Cpt = np.random.randint(5, 20, size=(P, T))  # Cost for patient assignments
# Drt = np.random.randint(10, 30, size=(R, T))  # Cost for resource allocation
# Ppt = np.random.randint(15, 25, size=(P, T))  # Penalty for unserved patients
# Oi = np.random.randint(10, 50, size=N)       # Overtime cost for staff
# Hi = np.random.randint(4, T + 1, size=N)     # Max working hours for staff
# Dt = np.random.randint(2, N // 2 + 1, size=T) # Demand for staff per time slot
# B = np.random.randint(500, 1000)             # Budget

# # Save data to CSV files
# pd.DataFrame(Wit, columns=[f"t{t+1}" for t in range(T)], index=[f"i{i+1}" for i in range(N)]).to_csv("Wit.csv")
# pd.DataFrame(Cpt, columns=[f"t{t+1}" for t in range(T)], index=[f"p{p+1}" for p in range(P)]).to_csv("Cpt.csv")
# pd.DataFrame(Drt, columns=[f"t{t+1}" for t in range(T)], index=[f"r{r+1}" for r in range(R)]).to_csv("Drt.csv")
# pd.DataFrame(Ppt, columns=[f"t{t+1}" for t in range(T)], index=[f"p{p+1}" for p in range(P)]).to_csv("Ppt.csv")
# pd.DataFrame({"Oi": Oi}, index=[f"i{i+1}" for i in range(N)]).to_csv("Oi.csv")
# pd.DataFrame({"Hi": Hi}, index=[f"i{i+1}" for i in range(N)]).to_csv("Hi.csv")
# pd.DataFrame({"Dt": Dt}, index=[f"t{t+1}" for t in range(T)]).to_csv("Dt.csv")
# pd.DataFrame({"B": [B]}).to_csv("Budget.csv")